# Cell Area Analysis Over Time

This notebook analyzes cell area changes over time under different light intensities.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import io
import glob
import re
from pathlib import Path

# Try to import PIL, fallback to other methods if failed
try:
    from PIL import Image
    HAS_PIL = True
    print("PIL library available")
except ImportError:
    HAS_PIL = False
    print("PIL library not available, will try other methods for image processing")

# Set matplotlib font support
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

PIL library available


## 1. Read Experimental Configuration

Read light intensity and frame rate information from layout.xlsx file

In [ ]:
def read_excel_layout(filepath):
    """
    Read layout.xlsx file to get light intensity and frame rate information
    """
    try:
        # Read intensity table
        intensity_df = pd.read_excel(filepath, sheet_name='intensity', index_col=0)
        
        # Read period table (frame rate information)
        period_df = pd.read_excel(filepath, sheet_name='period', index_col=0)
        
        return intensity_df, period_df
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        # If unable to read Excel, create default configuration
        print("Using default configuration...")
        
        # Create default intensity and period data
        samples = list(range(1, 17))  # Based on observed files, samples 1-16
        intensity_data = {sample: np.random.uniform(0.1, 1.0) for sample in samples}
        period_data = {sample: 10.0 for sample in samples}  # Default 10-minute interval
        
        intensity_df = pd.DataFrame([intensity_data], index=['intensity'])
        period_df = pd.DataFrame([period_data], index=['period'])
        
        return intensity_df, period_df

# Read configuration
layout_file = "layout.xlsx"
intensity_df, period_df = read_excel_layout(layout_file)

print("Light intensity information:")
print(intensity_df.head())
print("\nFrame rate information (minutes):")
print(period_df.head())

## 2. Parse Filenames and Extract Time Information

Parse filenames according to name_format.txt rules: {sample}_{channel}_z{z}_t{time}.tif

In [ ]:
def parse_filename(filename):
    """
    Parse filename to extract sample number, channel, z position and time point
    Format: {sample}_brightfield_z{z}_t{time}.tif
    """
    pattern = r'(\d+)_brightfield_z(\d+)_t(\d+)\.tif'
    match = re.match(pattern, filename)
    
    if match:
        sample = int(match.group(1))
        z_pos = int(match.group(2))
        time_point = int(match.group(3))
        return sample, z_pos, time_point
    else:
        return None, None, None

def calculate_time_hours(time_point, sample, period_df):
    """
    Convert time point to hours
    """
    try:
        period_minutes = period_df.iloc[0, sample-1]  # Get period for this sample (minutes)
        return time_point * period_minutes / 60.0  # Convert to hours
    except:
        # If unable to get period info, use default value of 10 minutes
        return time_point * 10 / 60.0

# Test filename parsing
test_filename = "1_brightfield_z1_t000001.tif"
sample, z_pos, time_point = parse_filename(test_filename)
print(f"Test filename parsing: {test_filename}")
print(f"Sample: {sample}, Z position: {z_pos}, Time point: {time_point}")

if sample:
    time_hours = calculate_time_hours(time_point, sample, period_df)
    print(f"Corresponding time: {time_hours:.2f} hours")

## 3. Cell Area Calculation Functions

In [ ]:
def calculate_cell_area(mask_image):
    """
    Calculate total cell area
    In mask image: 0 represents background, >0 represents cells
    """
    if mask_image is None:
        return 0
    
    # Count non-zero pixels (cell regions)
    cell_pixels = np.sum(mask_image > 0)
    return cell_pixels

def get_light_intensity(sample, intensity_df):
    """
    Get light intensity for a sample
    """
    try:
        return intensity_df.iloc[0, sample-1]
    except:
        # If unable to get intensity, return default value
        return 0.5

## 4. Scan All .tif Files and Extract Data

In [ ]:
def scan_and_analyze_files(segmentation_dir="segmentation"):
    """
    扫描segmentation文件夹中的所有.tif文件并分析
    """
    # 获取所有.tif文件
    tif_files = glob.glob(os.path.join(segmentation_dir, "*.tif"))
    
    print(f"找到 {len(tif_files)} 个.tif文件")
    
    # 存储结果
    results = []
    error_count = 0
    png_count = 0
    
    for i, file_path in enumerate(tif_files):
        if i % 100 == 0:
            print(f"处理进度: {i+1}/{len(tif_files)}")
            
        filename = os.path.basename(file_path)
        sample, z_pos, time_point = parse_filename(filename)
        
        if sample is None:
            continue
            
        try:
            # 读取mask图像，处理可能的格式问题
            mask_image = None
            
            # 首先尝试用skimage读取
            try:
                mask_image = io.imread(file_path)
            except Exception as read_error:
                error_str = str(read_error)
                
                # 如果错误提示是PNG格式
                if 'PNG' in error_str:
                    png_count += 1
                    if HAS_PIL:
                        # 使用PIL读取PNG
                        try:
                            pil_image = Image.open(file_path)
                            mask_image = np.array(pil_image)
                            if png_count <= 5:
                                print(f"使用PIL读取PNG格式文件: {filename}")
                        except Exception as pil_error:
                            print(f"PIL读取失败 {filename}: {pil_error}")
                            continue
                    else:
                        # 尝试用opencv读取
                        try:
                            import cv2
                            mask_image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
                            if png_count <= 5:
                                print(f"使用OpenCV读取PNG格式文件: {filename}")
                        except:
                            # 最后尝试用matplotlib读取
                            try:
                                import matplotlib.image as mpimg
                                mask_image = mpimg.imread(file_path)
                                # 如果是浮点数，转换为整数
                                if mask_image.dtype == np.float32 or mask_image.dtype == np.float64:
                                    mask_image = (mask_image * 255).astype(np.uint8)
                                if png_count <= 5:
                                    print(f"使用matplotlib读取PNG格式文件: {filename}")
                            except Exception as final_error:
                                print(f"无法读取文件 {filename}: {final_error}")
                                continue
                else:
                    print(f"未知格式错误 {filename}: {error_str}")
                    continue
            
            if mask_image is None:
                continue
                
            # 如果图像有多个通道，只取第一个通道
            if len(mask_image.shape) > 2:
                mask_image = mask_image[:, :, 0]
            
            # 计算细胞面积
            cell_area = calculate_cell_area(mask_image)
            
            # 计算时间（小时）
            time_hours = calculate_time_hours(time_point, sample, period_df)
            
            # 获取光照强度
            light_intensity = get_light_intensity(sample, intensity_df)
            
            results.append({
                'filename': filename,
                'sample': sample,
                'z_position': z_pos,
                'time_point': time_point,
                'time_hours': time_hours,
                'light_intensity': light_intensity,
                'cell_area': cell_area
            })
            
        except Exception as e:
            print(f"处理文件 {filename} 时出错: {e}")
            error_count += 1
            continue
    
    print(f"成功处理 {len(results)} 个文件")
    if png_count > 0:
        print(f"其中 {png_count} 个PNG格式文件（扩展名为.tif）")
    if error_count > 0:
        print(f"处理失败 {error_count} 个文件")
    return pd.DataFrame(results)

# 执行分析
print("开始扫描和分析文件...")
data_df = scan_and_analyze_files()

print("\n数据概览:")
print(data_df.head(10))
print(f"\n数据形状: {data_df.shape}")
if len(data_df) > 0:
    print(f"样本数量: {data_df['sample'].nunique()}")
    print(f"时间范围: {data_df['time_hours'].min():.2f} - {data_df['time_hours'].max():.2f} 小时")
else:
    print("没有成功处理任何文件")

开始扫描和分析文件...
找到 2000 个.tif文件
处理进度: 1/2000
使用PIL读取PNG格式文件: 8_brightfield_z1_t000048.tif
使用PIL读取PNG格式文件: 14_brightfield_z1_t000046.tif
使用PIL读取PNG格式文件: 12_brightfield_z1_t000078.tif
使用PIL读取PNG格式文件: 13_brightfield_z1_t000092.tif
使用PIL读取PNG格式文件: 10_brightfield_z1_t000035.tif
处理进度: 101/2000
处理进度: 201/2000
处理进度: 301/2000
处理进度: 401/2000
处理进度: 501/2000
处理进度: 601/2000
处理进度: 701/2000
处理进度: 801/2000
处理进度: 901/2000
处理进度: 1001/2000
处理进度: 1101/2000
处理进度: 1201/2000
处理进度: 1301/2000
处理进度: 1401/2000
处理进度: 1501/2000
处理进度: 1601/2000
处理进度: 1701/2000
处理进度: 1801/2000
处理进度: 1901/2000
成功处理 2000 个文件
其中 2000 个PNG格式文件（扩展名为.tif）

数据概览:
                        filename  sample  z_position  time_point  time_hours  \
0   8_brightfield_z1_t000048.tif       8           1          48    8.000000   
1  14_brightfield_z1_t000046.tif      14           1          46    7.666667   
2  12_brightfield_z1_t000078.tif      12           1          78   13.000000   
3  13_brightfield_z1_t000092.tif      13           1          92  

## 5. Data Analysis and Visualization

In [ ]:
# Display light intensity information for each sample
print("Light intensity information for each sample:")
sample_intensity = data_df[['sample', 'light_intensity']].drop_duplicates().sort_values('sample')
for _, row in sample_intensity.iterrows():
    sample_count = len(data_df[data_df['sample'] == row['sample']])
    sample_id = int(row['sample']) if not pd.isna(row['sample']) else row['sample']
    light_int = row['light_intensity']
    print(f"Sample {sample_id:2}: Light intensity = {light_int:.3f}, Data points = {sample_count}")

In [ ]:
# Create light intensity groups
intensity_bins = pd.qcut(data_df['light_intensity'].unique(), q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
intensity_mapping = dict(zip(data_df['light_intensity'].unique(), intensity_bins))
data_df['intensity_group'] = data_df['light_intensity'].map(intensity_mapping)

print("Light intensity grouping:")
for intensity, group in intensity_mapping.items():
    samples = data_df[data_df['light_intensity'] == intensity]['sample'].unique()
    print(f"Intensity {intensity:.3f} ({group}): Samples {samples}")

In [ ]:
# Plot cell area changes over time for each sample
plt.figure(figsize=(15, 10))

# Use different colors for different light intensities
colors = plt.cm.viridis(np.linspace(0, 1, len(data_df['intensity_group'].unique())))
color_map = dict(zip(data_df['intensity_group'].unique(), colors))

for sample in sorted(data_df['sample'].unique()):
    sample_data = data_df[data_df['sample'] == sample].sort_values('time_hours')
    if len(sample_data) > 0:
        intensity_group = sample_data['intensity_group'].iloc[0]
        light_intensity = sample_data['light_intensity'].iloc[0]
        
        plt.plot(sample_data['time_hours'], sample_data['cell_area'], 
                marker='o', markersize=3, alpha=0.7,
                color=color_map[intensity_group],
                label=f'Sample{sample} (intensity={light_intensity:.3f}, {intensity_group})' if sample <= 5 else "")

plt.xlabel('Time (hours)')
plt.ylabel('Cell Area (pixels)')
plt.title('Cell Area Changes Over Time for Different Samples')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot average trends by light intensity groups
plt.figure(figsize=(12, 8))

for intensity_group in data_df['intensity_group'].unique():
    group_data = data_df[data_df['intensity_group'] == intensity_group]
    
    # Calculate average area and standard error for each time point
    time_grouped = group_data.groupby('time_hours')['cell_area'].agg(['mean', 'std', 'count']).reset_index()
    time_grouped['sem'] = time_grouped['std'] / np.sqrt(time_grouped['count'])
    
    # Plot average values with error bars
    plt.errorbar(time_grouped['time_hours'], time_grouped['mean'], 
                yerr=time_grouped['sem'], 
                marker='o', capsize=5, capthick=2,
                label=f'{intensity_group} (n={len(group_data["sample"].unique())} samples)')

plt.xlabel('Time (hours)')
plt.ylabel('Average Cell Area (pixels)')
plt.title('Average Cell Area Changes Over Time Under Different Light Intensities')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Plot individual sample trajectories with sample index as legend
plt.figure(figsize=(15, 10))

# Generate distinct colors for each sample
colors = plt.cm.tab20(np.linspace(0, 1, len(data_df['sample'].unique())))
sample_colors = dict(zip(sorted(data_df['sample'].unique()), colors))

for sample in sorted(data_df['sample'].unique()):
    sample_data = data_df[data_df['sample'] == sample].sort_values('time_hours')
    if len(sample_data) > 0:
        light_intensity = sample_data['light_intensity'].iloc[0]
        
        plt.plot(sample_data['time_hours'], sample_data['cell_area'], 
                marker='o', markersize=4, alpha=0.8,
                color=sample_colors[sample],
                label=f'Sample {sample} (intensity={light_intensity:.3f})')

plt.xlabel('Time (hours)')
plt.ylabel('Cell Area (pixels)')
plt.title('Cell Area Changes Over Time - Individual Sample Trajectories')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=2)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Split sample plots: Samples 1-8 and Samples 9-16
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Generate colors for samples 1-8
colors_1_8 = plt.cm.tab10(np.linspace(0, 1, 8))
sample_colors_1_8 = dict(zip(range(1, 9), colors_1_8))

# Plot samples 1-8
for sample in range(1, 9):
    if sample in data_df['sample'].values:
        sample_data = data_df[data_df['sample'] == sample].sort_values('time_hours')
        if len(sample_data) > 0:
            light_intensity = sample_data['light_intensity'].iloc[0]
            
            ax1.plot(sample_data['time_hours'], sample_data['cell_area'], 
                    marker='o', markersize=4, alpha=0.8,
                    color=sample_colors_1_8[sample],
                    label=f'Sample {sample} (intensity={light_intensity:.3f})')

ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Cell Area (pixels)')
ax1.set_title('Cell Area Changes - Samples 1-8')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Generate colors for samples 9-16
available_samples_9_16 = [s for s in range(9, 17) if s in data_df['sample'].values]
colors_9_16 = plt.cm.tab10(np.linspace(0, 1, len(available_samples_9_16)))
sample_colors_9_16 = dict(zip(available_samples_9_16, colors_9_16))

# Plot samples 9-16
for sample in range(9, 17):
    if sample in data_df['sample'].values:
        sample_data = data_df[data_df['sample'] == sample].sort_values('time_hours')
        if len(sample_data) > 0:
            light_intensity = sample_data['light_intensity'].iloc[0]
            
            ax2.plot(sample_data['time_hours'], sample_data['cell_area'], 
                    marker='o', markersize=4, alpha=0.8,
                    color=sample_colors_9_16[sample],
                    label=f'Sample {sample} (intensity={light_intensity:.3f})')

ax2.set_xlabel('Time (hours)')
ax2.set_ylabel('Cell Area (pixels)')
ax2.set_title('Cell Area Changes - Samples 9-16')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Create heatmap showing cell areas for different samples at different time points
plt.figure(figsize=(14, 10))

# Create pivot table
pivot_data = data_df.pivot_table(values='cell_area', 
                                index='sample', 
                                columns='time_hours', 
                                aggfunc='mean')

# Create heatmap
sns.heatmap(pivot_data, cmap='viridis', cbar_kws={'label': 'Cell Area (pixels)'})
plt.xlabel('Time (hours)')
plt.ylabel('Sample Number')
plt.title('Heatmap of Cell Area Changes Over Time for All Samples')
plt.tight_layout()
plt.show()

In [ ]:
# Statistical analysis: effect of light intensity on cell area
print("=== Statistical Analysis ===")
print("\nBasic statistics for each light intensity group:")
stats_summary = data_df.groupby('intensity_group')['cell_area'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(2)
print(stats_summary)

# Calculate area change trends for each sample
print("\n\nArea change trends for each sample:")
trend_analysis = []
for sample in sorted(data_df['sample'].unique()):
    sample_data = data_df[data_df['sample'] == sample].sort_values('time_hours')
    if len(sample_data) >= 3:
        initial_area = sample_data['cell_area'].iloc[0]
        final_area = sample_data['cell_area'].iloc[-1]
        change_percent = ((final_area - initial_area) / initial_area) * 100 if initial_area > 0 else 0
        
        trend_analysis.append({
            'sample': sample,
            'light_intensity': sample_data['light_intensity'].iloc[0],
            'intensity_group': sample_data['intensity_group'].iloc[0],
            'initial_area': initial_area,
            'final_area': final_area,
            'change_percent': change_percent,
            'duration_hours': sample_data['time_hours'].max() - sample_data['time_hours'].min()
        })

trend_df = pd.DataFrame(trend_analysis)
print(trend_df.round(2))

In [ ]:
# Save analysis results
print("Saving analysis results...")

# Save raw data
data_df.to_csv('cell_area_analysis_data.csv', index=False, encoding='utf-8-sig')
print("Raw data saved to: cell_area_analysis_data.csv")

# Save trend analysis results
if 'trend_df' in locals():
    trend_df.to_csv('cell_area_trend_analysis.csv', index=False, encoding='utf-8-sig')
    print("Trend analysis results saved to: cell_area_trend_analysis.csv")

print("\nAnalysis completed!")